In [18]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ezc3d import c3d
from scipy.signal import resample
from scipy.io import loadmat, savemat
from joblib import load, dump
from tqdm import tqdm

In [19]:
PATH_DATASETS = '../../Dataset/'
PATH_DATASET_1 = 'gait-dbase-1/'
PATH_DATASET_2 = 'gait-dbase-2/'

N_SAMPLES = 1024

In [20]:
path = os.path.join(PATH_DATASETS, PATH_DATASET_1)
subjects = os.listdir(path)

features = pd.DataFrame()
target = pd.DataFrame()

for subject in tqdm(subjects):
    files_path = os.path.join(path, subject)
    files = os.listdir(files_path)
    for filename in files:

        try:
            content = c3d(os.path.join(path, subject, filename), extract_forceplat_data=True)
            trajectory_x = content['data']['points'][1, 9:15, :]
            trajectory_y = content['data']['points'][1, 9:15, :]
            trajectory_z = content['data']['points'][2, 9:15, :]

            trajectory_x = resample(trajectory_x, N_SAMPLES, axis=1).T
            trajectory_y = resample(trajectory_y, N_SAMPLES, axis=1).T
            trajectory_z = resample(trajectory_z, N_SAMPLES, axis=1).T
            
            _features = pd.DataFrame(
                np.concatenate([trajectory_x, trajectory_y, trajectory_z], axis=1),
                columns=[
                    'lfal_x', 'ltam_x', 'lfcc_x', 'lfm1_x', 'lfm2_x', 'lfm5_x',
                    'lfal_y', 'ltam_y', 'lfcc_y', 'lfm1_y', 'lfm2_y', 'lfm5_y',
                    'lfal_z', 'ltam_z', 'lfcc_z', 'lfm1_z', 'lfm2_z', 'lfm5_z'
                ]
            )

            force_x = content['data']['platform'][0]['force'][0, :]
            force_y = content['data']['platform'][0]['force'][1, :]
            force_z = content['data']['platform'][0]['force'][2, :]

            force_x = resample(force_x, N_SAMPLES, axis=0).T
            force_y = resample(force_y, N_SAMPLES, axis=0).T
            force_z = resample(force_z, N_SAMPLES, axis=0).T

            _target = pd.DataFrame(
                np.stack([force_x, force_y, force_z], axis=1),
                columns=['fx', 'fy', 'fz']
            )

            features = pd.concat([features, _features], axis=0)
            target = pd.concat([target, _target], axis=0)

        except Exception as e:
            # print('Error occured', e)
            pass



100%|██████████| 50/50 [03:47<00:00,  4.55s/it]


In [21]:
features

,lfal_x,ltam_x,lfcc_x,lfm1_x,lfm2_x,lfm5_x,lfal_y,ltam_y,lfcc_y,lfm1_y,lfm2_y,lfm5_y,lfal_z,ltam_z,lfcc_z,lfm1_z,lfm2_z,lfm5_z
0,315.077850,229.390457,266.381287,249.976181,307.238617,358.050293,315.077850,229.390457,266.381287,249.976181,307.238617,358.050293,75.325401,92.450508,39.130699,22.079981,35.831905,21.600080
1,312.579152,226.591457,264.572781,247.903697,305.345070,357.181279,312.579152,226.591457,264.572781,247.903697,305.345070,357.181279,76.706277,92.092801,41.526899,17.051512,31.799598,19.159787
2,311.619596,225.518568,263.876385,247.135499,304.630233,356.808721,311.619596,225.518568,263.876385,247.135499,304.630233,356.808721,77.493047,92.190563,42.817798,15.211522,30.329800,18.300263
3,311.968866,225.912648,264.126853,247.473506,304.915576,356.859162,311.968866,225.912648,264.126853,247.473506,304.915576,356.859162,77.765488,92.664582,43.162852,16.052687,31.010942,18.761375
4,313.162473,227.252454,264.989597,248.522765,305.844398,357.182654,313.162473,227.252454,264.989597,248.522765,305.844398,357.182654,77.704683,93.378374,42.899788,18.604518,33.062358,20.060689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,346.628700,441.376485,398.686245,440.067989,383.182452,335.770303,346.628700,441.376485,398.686245,440.067989,383.182452,335.770303,63.855626,87.560454,6.623271,65.443163,81.658080,52.203568
1020,343.302850,437.967856,395.244875,435.861783,379.069766,333.169987,343.302850,437.967856,395.244875,435.861783,379.069766,333.169987,63.235123,86.861761,3.293285,69.670258,85.333197,54.400806
1021,341.827737,436.486933,393.664424,434.247998,377.510917,332.258755,341.827737,436.486933,393.664424,434.247998,377.510917,332.258755,62.703167,86.230001,1.749628,70.790194,86.198857,54.647374
1022,348.049370,443.021523,399.780983,443.369238,386.527415,338.373009,348.049370,443.021523,399.780983,443.369238,386.527415,338.373009,62.458395,85.791040,7.331997,59.255723,75.659870,47.002035


In [22]:
target

,fx,fy,fz
0,-0.001750,-0.008617,-0.124157
1,0.001825,0.008875,0.128063
2,-0.001904,-0.009149,-0.132206
3,0.001989,0.009440,0.136609
4,-0.002079,-0.009750,-0.141296
...,...,...,...
1019,-0.000648,-0.000282,0.026254
1020,0.000655,0.000286,-0.026192
1021,-0.000662,-0.000290,0.026132
1022,0.000669,0.000294,-0.026072


In [23]:
path = '../../Dataset/Processed/data.joblib'
data = {
    'X': features,
    'y': target
}
dump(data, path)

['../../Dataset/Processed/data.joblib']